## 实时量化推理demo
-------

In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import time
import numpy as np
import random

In [2]:
device =  'cpu'
config = {
    'seed': 42,   
    'valid_ratio': 0.3,
    'n_epochs': 10,        
    'batch_size': 64, 
    'learning_rate': 5e-3,              
    'early_stop': 3,    
    'save_path': './models/model.ckpt',  # model will be saved here.
    'data_path': './DataSet/npz/',
    'data_file': ["cat_eval","cat_train",
                  "dog_eval", "dog_train",
                  "other_eval","other_train"],
    'backend':'fbgemm',
}

ENV = 'REAL'

### 加载模型

In [3]:
class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bias=False):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels,in_channels,kernel_size,groups=in_channels,padding=1)
        self.pointwise = nn.Conv2d(in_channels,out_channels,kernel_size=kernel_size,padding=1)
        self.outlayer = nn.ReLU()
    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        out = self.outlayer(out)
        return out
    
class myConv2d(nn.Module):
    def __init__(self,input_channels,output_channels,kernel_size,dense_dim,bn_dim):
        super(myConv2d, self).__init__()
        self.conv2d = nn.Sequential(
            nn.Conv2d(input_channels,output_channels,kernel_size,padding=1),
            nn.BatchNorm2d(bn_dim),
            nn.ReLU()
        )
        self.spconv2d1 = nn.Sequential(
            SeparableConv2d(output_channels,32,kernel_size),
            nn.MaxPool2d((2,2)),
            nn.Dropout(0.2)
        )
        self.spconv2d2 = nn.Sequential(
            SeparableConv2d(32,output_channels,kernel_size),
            nn.MaxPool2d((2,2)),
            nn.Dropout(0.2)
        )
        self.outlayer = nn.Sequential(
            nn.Linear(dense_dim,output_channels),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
    def forward(self,x):
#         print(x.shape)
        out = self.conv2d(x)
#         print(out.shape)
        out = self.spconv2d1(out)
#         print(out.shape)
        out = self.spconv2d2(out)
#         print(out.shape)
        out = torch.flatten(out,start_dim=1)
#         print(out.shape)
        out = self.outlayer(out)
#         print(out.shape)
        return out
    
class Tiny2Net(nn.Module):
    def __init__(self, labels):
        super(Tiny2Net, self).__init__()
#         self.args = args
        self.videoNet = myConv2d(3,64,(3,3),4096,64)  #(3,64,(3,3),4096,32)
        self.audioNet = myConv2d(1,64,(3,3),2112,64) #(1,64,(3,3),2112,44)
        self.layer1 = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(64,labels),
            nn.Softmax(dim=0)
        )
    def forward(self,x,y):
        """
        input x   MFCC Vector     size:  44x13x1
        input y   Image Vector   size: 32x32x3
        """
        x = self.audioNet(x)
        y = self.videoNet(y)
        z = torch.cat((x,y),1)
#         print("z:",z.shape)
        z = self.layer1(z)
#         print("z:",z.shape)
        z = self.layer2(z)
        return z

In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
same_seed(config['seed'])

In [5]:
model_fp32 = Tiny2Net(3)
state_dict = torch.load(r'./models/seed_42_1205_073217_entropy_standardnoise_.pth')
model_fp32.load_state_dict(state_dict)


model_int8 = torch.quantization.quantize_dynamic(
    model_fp32,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights


### 加载数据

In [6]:
ENV = 'DEBUG' #非实时输入状态下进行测试
# from CameraAudioRead.AudioRead import read_audio
# from CameraAudioRead.CameraRead import read_camera

In [7]:
def loadData(path,dataList):
    def getName(base,file):
        return base+file+'.npz'
    dataset={}
    for item in dataList:
        dataset[item]=np.load(getName(path,item))
    return dataset

_x = None
_y = None
_z = None

if ENV == 'DEBUG':
    dataset = loadData(config['data_path'],config['data_file'])
    for k,v in dataset.items():
        if _x is None:
            _x = v["x"]
            _y = v["y"]
            _z = v["z"]
        else:
            _x = np.concatenate((_x, v["x"]), axis=0)
            _y = np.concatenate((_y, v["y"]), axis=0)
            _z = np.concatenate((_z, v["z"]), axis=0)

In [8]:
def load_data_item():
    if ENV =='DEBUG':
        idx = random.randint(0, _x.shape[0])
        return torch.from_numpy(_x[idx]).unsqueeze(0),torch.from_numpy(_y[idx]).unsqueeze(0),torch.from_numpy(_z[idx]).unsqueeze(0)
#     else:
#         camera_frames = read_camera(resize_h=32, resize_w=32, frames_number=1)
#         audio_frames = read_audio(record_second=1,frames_number=1)
#         y = torch.from_numpy(camera_frames[0]).unsqueeze(0)
#         x = torch.from_numpy(audio_frames[0]).unsqueeze(0)
#         reutrn x,y,None

##  运行

In [9]:
run_time = 1

criterion = nn.MSELoss(reduction='mean')
criterion = nn.CrossEntropyLoss()
animal_dict = {0:"cat",1:"dog",2:"other animal"}
while(run_time>=0):
    x,y,z=load_data_item()
    x=x.to(device)
    y=y.to(device)
    print(x.shape)
    print(y.shape)
    with torch.no_grad():
        pred_quantized = model_int8(x.float(), y.float())
        
        target_list_quantized = pred_quantized.cpu().numpy().tolist()[0]
        target_quantized = target_list_quantized.index(max(target_list_quantized))
        print("Quantized:the prediction is {} ,corresponding animal is {}".format(target_quantized,animal_dict[target_quantized]))
    print(run_time)
    run_time=run_time-1

torch.Size([1, 1, 44, 13])
torch.Size([1, 3, 32, 32])
Quantized:the prediction is 0 ,corresponding animal is cat
1
torch.Size([1, 1, 44, 13])
torch.Size([1, 3, 32, 32])
Quantized:the prediction is 0 ,corresponding animal is cat
0
